In [133]:
import os
import tempfile
from langchain.document_loaders import PyPDFLoader, UnstructuredEPubLoader

def load_book(file_obj, file_extension):
    """Load the content of a book based on its file type."""
    text = ""
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_file:
        temp_file.write(file_obj.read())
        if file_extension == ".pdf":
            loader = PyPDFLoader(temp_file.name)
            pages = loader.load()
            text = "".join(page.page_content for page in pages)
        elif file_extension == ".epub":
            loader = UnstructuredEPubLoader(temp_file.name)
            data = loader.load()
            text = "\n".join(element.page_content for element in data)
        else:
            raise ValueError(f"Unsupported file extension: {file_extension}")
        os.remove(temp_file.name)
    text = text.replace('\t', ' ')
    return text

In [134]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

def split_and_embed(text, openai_api_key):
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=10000, chunk_overlap=3000)
    docs = text_splitter.create_documents([text])
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectors = embeddings.embed_documents([x.page_content for x in docs])
    return docs, vectors

In [135]:
from sklearn.cluster import KMeans
import numpy as np

def cluster_embeddings(vectors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)
    closest_indices = [np.argmin(np.linalg.norm(vectors - center, axis=1)) for center in kmeans.cluster_centers_]
    return sorted(closest_indices)

In [136]:
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

def summarize_chunks(docs, selected_indices, openai_api_key):
    llm3_turbo = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=1000, model='gpt-3.5-turbo-16k')
    map_prompt = """
    这里有一段来自一本书的段落。你的任务是对这段文字进行全面的总结。确保准确性，避免添加任何不在原文中的解释或额外细节。摘要至少应有三段，完整地表达出原文的要点。
    ```{text}```
    总结:
    """
    map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])
    selected_docs = [docs[i] for i in selected_indices]
    summary_list = []

    for doc in selected_docs:
        chunk_summary = load_summarize_chain(llm=llm3_turbo, chain_type="stuff", prompt=map_prompt_template).run([doc])
        summary_list.append(chunk_summary)
    
    return "\n".join(summary_list)

In [137]:
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI

def create_final_summary(summaries, openai_api_key):
    llm4 = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=3000, model='gpt-4', request_timeout=120)
    combine_prompt = """
    这里有一些段落摘要，它们来自于一本书。你的任务是把这些摘要编织成一个连贯且详细的总结。读者应该能够从你的总结中理解书中的主要事件或要点。确保保持内容的准确性，并以清晰而引人入胜的方式呈现。
    ```{text}```
    综合总结:
    """
    combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])
    reduce_chain = load_summarize_chain(llm=llm4, chain_type="stuff", prompt=combine_prompt_template)
    final_summary = reduce_chain.run([Document(page_content=summaries)])
    return final_summary

In [138]:
def generate_summary(uploaded_file, openai_api_key, num_clusters=11, verbose=False):
    file_extension = os.path.splitext(uploaded_file.name)[1].lower()
    text = load_book(uploaded_file, file_extension)
    docs, vectors = split_and_embed(text, openai_api_key)
    selected_indices = cluster_embeddings(vectors, num_clusters)
    summaries = summarize_chunks(docs, selected_indices, openai_api_key)
    final_summary = create_final_summary(summaries, openai_api_key)
    return final_summary

In [139]:
# Testing the summarizer
if __name__ == '__main__':
    openai_api_key = os.getenv('OPENAI_API_KEY')
    # book_path = "./thethreekingdoms.pdf"
    book_path = './wentiejun.pdf'
    with open(book_path, 'rb') as uploaded_file:
        summary = generate_summary(uploaded_file, openai_api_key, verbose=True)
        print(summary)

/Users/huke/openai-learning/venv2405/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
/Users/huke/openai-learning/venv2405/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/huke/

这本书主要探讨了中国在过去几十年中所经历的八次经济危机，包括其原因、影响以及解决策略。首先，作者讨论了1958年至1976年期间的三次危机，这些危机主要源于国家工业化和城市化进程中的制度成本转嫁到农村。其次，1978年至1997年期间的三次内源性经济危机，主要由改革开放政策引发的经济结构调整和外资引入所导致。最后，作者提到了中国在1993年至1994年期间以及1997年和2008年面临的危机，这些危机主要是由于外债和财政赤字的增加以及经济过热所引发的。

在这个过程中，中国采取了一系列的应对策略。例如，20世纪60年代至70年代，中国实施了“三线建设”战略，将工业布局从沿海城市转移到内陆和西部地区，以应对国际环境的威胁。然而，这种战略导致了严重的财政赤字和经济危机。在1979年至1981年的经济危机中，政府采取了一系列的补贴和福利政策来缓解危机。然而，这些措施并没有解决投资规模过大的问题，导致经济危机的持续发展。

在20世纪80年代和90年代，中国经历了经济结构的转变，从内向型转向外向型。这一转变带来了新的问题，如外汇储备的下降和对外依存度的增加。同时，经济危机也对农村产生了深远影响，包括农民收入的提高和农民进城务工的增加，以及粮食供给过剩和农业经营收入下降等问题。

此外，作者还讨论了中国面临的一些挑战，包括全球化趋势中的法西斯化、中美战略接近关系以及发展中国家劳工群体的困境。作者认为，中国需要面对这些挑战并制定相应的应对策略。同时，中国也需要解决一些内部问题，如灰色经济和收入差距、政治体制改革以及国际形势等。

总的来说，这本书通过对中国经济危机的深入分析，提出了一些关于中国经济发展的观点和建议，对于理解中国经济的发展历程和未来挑战具有重要的参考价值。
